In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [38]:
#test
m = nn.Linear(2,3)
in_ = torch.randn(4,2)
out_ = m(in_)


torch.randn(10)  > 0

tensor([1, 0, 1, 1, 1, 0, 1, 0, 0, 0], dtype=torch.uint8)

In [42]:
A = torch.tensor([1,2,2])
torch.cumsum(A,dim = 0)

tensor([1, 3, 5])

In [20]:
class Rmtpp(nn.Module):
    
    def __init__(self,marker_dim):
        super(Rmtpp, self).__init__()
        #embedding yj for a more compact and efficient representation
        self.embedding = nn.Linear(1,1)
        #linear transformation
        self.lin_op = nn.linear(3,1) 
        self.vt = nn.linear(1,1)
        #weights
        self.w_t = torch.rand(1) 
        self.w = torch.rand(1) 
        self.V_y = torch.rand(marker_dim)
        self.b_y = torch.rand(marker_dim) #bias
        #functions
    
    #compute the function fstar
    def fstart(t,tj,hj):
        return torch.exp(self.vt(hj) + self.w_t*(t-tj) + 1/self.w * torch.exp(self.vt(hj)) -1/w * torch.exp(self.vt(hj) + self.w_t*(t-tj)))
    
    #compute integral of t*fstart(t) for tj to infinity using monteCarlo method
    def numerical_mean(tj,N = 10000):
        samp = torch.randn(N)
        sample = torch.sqrt(2*F.math.pi) * torch.exp(1/2 * samp**2) * samp * fstart(samp)
        #I select only value greater than tj
        sample = torch.dot(samp>tj,sample) 
        #then cumsum of sample 
        return torch.cumsum(sample,dim=0)
        
        
        
    def forward(self, tim"e, marker, hidden_state):
        #I first compute next time
        time = numerical_mean(time, N =10000)
        #Then next marker
        marker = ?
        #Finally next hidden_state
        embedding = self.embedding(marker)
        input_ = torch.cat(embedding, time, hidden_state)
        hidden_state = F.relu(self.lin_op(input_))
        
        
        
        return logprob, logfstar, hidden_state
        
    
        
    
        
        
        
        
        
        
        
        
        
        
        
        
        